In [ ]:
# 출처 : https://jxnjxn.tistory.com/49

In [ ]:
# !pip install glove_python_binary

In [263]:
# 한글정제 KoNLPy
from ckonlpy.tag import Twitter
twitter = Twitter()

#폴더 내용 불러오기
from glob import glob
import os 
import pandas as pd

# 시간기록
from datetime import datetime, timedelta
import time 

# 분석2 : wordcloud 준비
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)
from wordcloud import WordCloud


from gensim import corpora
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [230]:
df = pd.read_excel('./21년KPI.xlsx',0)

In [231]:
df.tail(2)

,사번,조직,직급,KPI선택,KPI
5338,507101,감사팀,P4,직접입력,누수구역 자체 개선 건수
5339,507101,감사팀,P4,직접입력,공사 완료 기간 내 단축 달성도


In [251]:
# 사전에 고유 명사 추가
twitter.add_dictionary('셀렉스', 'Noun')
twitter.add_dictionary('클레임', 'Noun')
twitter.add_dictionary('최소화', 'Noun')
twitter.add_dictionary('시행률', 'Noun')
twitter.add_dictionary('매출액', 'Noun')
twitter.add_dictionary('매출 액', 'Noun')
twitter.add_dictionary('재구매', 'Noun')
twitter.add_dictionary('안정성', 'Noun')
twitter.add_dictionary('달성율', 'Noun')
twitter.add_dictionary('직접이익', 'Noun')
twitter.add_dictionary('영업이익', 'Noun')
twitter.add_dictionary('미출', 'Noun')


def tokenize(doc):
    try:
        result = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)] # ex '더빙/Norm', '나다/Verb'
    except:
        result = ''
    return result 


def tokenize2(doc):
    try:
        result = [t1  for t1,t2 in okt.pos(doc, norm=True, stem=True)] # ex '더빙/Norm', '나다/Verb'
    except:
        result = ''
    return result 

def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()  
    return str1


In [252]:
# test
print(df_product['KPI'][0])
tokenize2(df_product['KPI'][0])

회계업무 프로세스 정립을 통한 재무제표 도출


['회계', '업무', '프로세스', '정립', '을', '통한', '재무제표', '도출']

In [253]:
#토큰화 : 전체리뷰자료
df_temp1 = df['KPI'].replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

# regex to konlpy
# token_all = [tokenize2(row) for row in df_temp1 ]  
token_all = [list(dict.fromkeys(tokenize2(row))) for row in df_temp1]

data_trim2 = []
for i in range(len(df)):
    t = ' '.join(token_all[i])
    data_trim2.append(t.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]",""))
    
df_temp1 = pd.DataFrame(data_trim2)
df_temp1 = df_temp1[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [254]:
df['token2'] = df_temp1
df['token1'] = token_all

In [255]:
# df.groupby('조직')['KPI'].count().head(10)

In [256]:
df.token2= df.token2.str.replace(' 액', '액')
df.token2= df.token2.str.replace(' 량', '량')
df.token2= df.token2.str.replace(' 율', '율')
df.token2= df.token2.str.replace('클레 임', '클레임')
df.token2= df.token2.str.replace('연 평균', '연 평균')
df.token2= df.token2.str.replace('최소 화', '최소화')
df.token2= df.token2.str.replace('최소 화', '최소화')
df.token2= df.token2.str.replace('영업 이익', '영업이익')

In [257]:
df[220:230]

,사번,조직,직급,KPI선택,KPI,token2,token1
220,512101,MC팀,P3,직접입력,목표 대비 달성율,목표 대비 달성율,"[목표, 대비, 달성, 율]"
221,512101,MC팀,P3,직접입력,강의 거래처수\n만족도점수(5점척도),강의 거래처 수 만족도 점수 점 척도,"[강의, 거래처, 수, \n, 만족도, 점수, (, 5, 점, 척도, )]"
222,809105,MC팀,P2,직접입력,목표대비 달성율 - 실적/목표 * 100%,목표 대비 달성율 실적,"[목표, 대비, 달성, 율, -, 실적, /, *, 100%]"
223,809105,MC팀,P2,직접입력,목표대비 달성율,목표 대비 달성율,"[목표, 대비, 달성, 율]"
224,809105,MC팀,P2,직접입력,목표대비 달성율,목표 대비 달성율,"[목표, 대비, 달성, 율]"
225,809105,MC팀,P2,직접입력,목표대비 달성율 - 실적/목표 * 100%,목표 대비 달성율 실적,"[목표, 대비, 달성, 율, -, 실적, /, *, 100%]"
226,809105,MC팀,P2,직접입력,강의 거래처수\n만족도점수(5점척도),강의 거래처 수 만족도 점수 점 척도,"[강의, 거래처, 수, \n, 만족도, 점수, (, 5, 점, 척도, )]"
227,903205,MC팀,P3,직접입력,목표대비 달성율,목표 대비 달성율,"[목표, 대비, 달성, 율]"
228,903205,MC팀,P3,직접입력,목표대비 달성율,목표 대비 달성율,"[목표, 대비, 달성, 율]"
229,903205,MC팀,P3,직접입력,강의 거래처수\n만족도점수(5점척도),강의 거래처 수 만족도 점수 점 척도,"[강의, 거래처, 수, \n, 만족도, 점수, (, 5, 점, 척도, )]"


In [258]:
x_data = listToString(df['token2'])
tokens = x_data.split(' ')
my_dict = nltk.FreqDist(tokens)
df2 = pd.DataFrame(list(my_dict.items()),columns = ['column1','column2']) 
# df2['column3'] = text
df2 = df2.sort_values(by='column2', ascending=False)

In [270]:
# def wordcloud_gen(dataset, color):
#     text = dataset.to_list()
#     text_all = listToString(text)
#     wc1 = WordCloud(font_path= path, stopwords = stopword, width = 600, height = 600, background_color="white", repeat=True, colormap = color)
#     return wc1.generate(text_all)


stopword = ['먹다', '하다', '좋다'  , '예요', '이에요', '있다', '맛있다', '같다', '없다', '보통', '최고', '주문', '보다', '매일',
         '구매', '에서', '먹이다', '않다', '자다'  , '들다', '사다'  , '되다', '으로', 'ㅠㅠ', 'ㅜㅜ', '말다', '이다', '받다',
         '아직', '이다', '보이다', '이나', '오다'  , '이라', '우유'  , '배달', '배송', '가정', '제품', '신청', '문의', '되어다',
         '넘다', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅎㅎ', '진짜'  , '너무', '이', '는', '요', '은', '에', '기', '다', '도', '원', '구', '개', '랑','가',
         '도', '임시', '이미지', '확인', '후', '을', '안' ,'되다', '하', '심', '고객', '님', '어요', '좀', '것', '니', '어보', '타',
         '생각', '지만', '잘', '제', '니다', '네', '해', '그래도', '엄청', '하고', '번', '그', '나다', '시', '다가', '한', '해서', '로',
         '더', '못', '에는', '던데', '일단', '나요', '아하', '이적','이고', '려고', '이라고', '고', '어서','를', '그냥', '만', '짧다',
         '유통기한', '인데', '박스', '까지', '일', '월', '두', '팩','어요', '요','다', '한', '있다', '라', '나', '게', '거', '데',
         '더니','달', '보', '이번', '이네', '내주', '하나', '수', '저', '이렇다', '이렇게', 'ㅠ', '때', '늘', '또', '편', '늘','감사',
         '항상', '여', '아해', '들', '중', '열', '영', '분', '세', '뭐', '루', '알', '용', '통', '함', '무', '생', '물', '리', '새',
         '적', 'hellip', '빠르다', '도착', '배송', '택배', '두유', '아주', '지', '아몬드브리즈', '아몬드 브리즈', '브리', '즈', '아주',
         '씩', '오리', '널', '에요', '에도', '네요','잇다', '알다', '포장', '옵션', '상품', '많이', '대하',
         '어도', '오전', '오후', '안녕', '인가요', '안되다','변경', '뜨다', '혹시', '언제','반품', '환불', '가능', '가요','번호', 
         '질문', '이메일','내용', '물건', '완료', '총' '건가', '빨', '어떻다', '오늘','건가', '주소', '연락', '부탁', '세요', '면' ,'총',
         '출고', '준비', '중이', '철회', '금요일', '바라다','아야', '라고', '리좀', '기사','드리다', '일부', '부터', '조회', '날', '서',
         '터', '답변', '시작', '드립', '취소', '발송', '수요일', '요청', '송장', '왜', '궁금하자', '쯤', '매일유업', '재', '입다', '해주',
         '돼다', '아니', '다시', '싶다', '명' , '연락처', '궁금하다', '주', '해주시', '회' ,'아니다', '자명', '상태', '건', '늘다', 
         '보내다','중인', '문자', '해도', '회수', '쇼핑', '편이', '에요', '여요' '꾸다', '라고', '이건', '다른', '면', '그렇다', '니까',
         '다고', '만족도' , '엔', '음번'
         ]

In [241]:
# plt.subplots_adjust(left=0.125, bottom=0.1,  right=0.9, top=0.9, wspace=0.2, hspace=0.3)
# plt.xticks([]); plt.yticks([]); plt.grid(False)
# plt.imshow(wordcloud_gen((df[df['조직']=='FS국내영업1팀']['token']), None), cmap=plt.cm.binary)
# plt.title('전체리뷰',fontsize='large')
# plt.show()

In [267]:
################
#한글 토큰화
#####

# 1차 가공 : koNLpy to token
def tokenize(doc):
    result = [t for t in okt.pos(doc, norm=True, stem=True)] # ex '더빙/Norm', '나다/Verb'
    return result 

token_data1 = [tokenize(row) for row in df['KPI']]  

In [268]:
token_data2 = []
for i in range(0, len(token_data1)):
    r = []
    for word in token_data1[i]:
        if not word[1] in ["Josa", "Punctuation", "Foreign", "Suffix", "Eomi"]:
            r.append(word[0])
    token_data2.append(r)

In [272]:
X_train = []
for sentence in token_data2:
    temp_X = [word for word in sentence if not word in stopword] # 불용어 제거
    X_train.append(temp_X)

In [273]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
            else:
                break
    return(topic_table)

In [284]:
# 사전 및 단어집 만들기
dictionary = corpora.Dictionary(X_train)
corpus = [dictionary.doc2bow(text) for text in X_train]

# 토픽 겟수 설정 : 20개
NUM_TOPICS = 20 

# LAD 모델링
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=16)
topics = ldamodel.print_topics(num_words=5)

In [285]:
topics

[(0, '0.116*"평균" + 0.090*"매장" + 0.070*"12월" + 0.066*"대비" + 0.059*"4월"'),
 (1, '0.074*"취득" + 0.057*"전" + 0.050*"자격증" + 0.043*"횟수" + 0.042*"년"'),
 (2, '0.094*"과제" + 0.083*"사업" + 0.049*"POS" + 0.045*"분석" + 0.044*"계획"'),
 (3, '0.072*"개인" + 0.045*"별" + 0.042*"점수" + 0.034*"개선" + 0.031*"사용"'),
 (4, '0.172*"FS" + 0.132*"기준" + 0.121*"MS" + 0.050*"백색" + 0.046*"달성"'),
 (5, '0.088*"및" + 0.079*"보고" + 0.068*"사항" + 0.067*"조직" + 0.047*"적발"'),
 (6, '0.226*"달성" + 0.223*"율" + 0.123*"매출" + 0.101*"목표" + 0.095*"액"'),
 (7, '0.133*"금액" + 0.082*"절감" + 0.074*"원가" + 0.072*"생산" + 0.048*"kg"'),
 (8, '0.058*"거래처" + 0.053*"유" + 0.047*"발효" + 0.044*"주요" + 0.032*"지점"'),
 (9, '0.310*"이익" + 0.192*"직접" + 0.137*"영업" + 0.037*"카테고리" + 0.027*"전환"'),
 (10, '0.083*"상하" + 0.068*"개선" + 0.057*"업무" + 0.042*"프로세스" + 0.041*"유기농"'),
 (11, '0.092*"준수" + 0.089*"율" + 0.068*"확대" + 0.060*"취급" + 0.047*"SKU"'),
 (12, '0.231*"개선" + 0.105*"대리점" + 0.101*"구조" + 0.036*"실행" + 0.030*"별"'),
 (13, '0.144*"타겟" + 0.112*"품질" + 0.109*"이슈" + 0.060*"이상" + 

In [ ]:
["보고서",
 (1, '0.255*"직접" + 0.079*"별" + 0.046*"확대" + 0.036*"건수" + 0.036*"전환"'),
 '영업이익율 달성",
 '신규 금액 투입',
 "FS/GT 실적",
 "MS 기준",
 "대리점 통합",
 '클레임',
 "일정 준수",
 "매장 프로그램 개발",
 '매출액 목표 달성',
 '상하 유기농 사업부',
 "100 이상 취득",
 "교육",
 "제안 활동",
 '개척 목표 달성',
 '대리점 구조 개선',
 "정성 평가 진행",
 "품질 이슈",
 "생산 원가 절감" ]

In [278]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']

# topictable['(code)구매제품'] = list(result0['(code)구매제품'])
# topictable['내용'] = list(result0['내용'])

In [280]:
topictable.head()

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,4.0,0.3813,"[(4, 0.3812588), (11, 0.25622952), (16, 0.1312..."
1,1,17.0,0.2625,"[(0, 0.012501147), (1, 0.012501147), (2, 0.012..."
2,2,11.0,0.2912,"[(3, 0.15252127), (4, 0.13465723), (9, 0.17129..."
3,3,16.0,0.3503,"[(0, 0.016682014), (1, 0.016682014), (2, 0.016..."
4,4,16.0,0.3500,"[(0, 0.016666884), (1, 0.016666884), (2, 0.016..."


In [283]:
# Excel 자료 저장 v2
now = datetime.today()    # 현재
date = now.strftime('%Y%m%d')

writer = pd.ExcelWriter('./'+ date+ '_KPI_v2.xlsx')
df.to_excel(writer, 'Sheet1')
df2.to_excel(writer, 'single')
topictable.to_excel(writer, 'topic')
writer.save()